# Criando um banco de dados relacionados aos Beatles usando a api de dados discogs

## Objetivo
Este é um exemplo prático utilizando a API de dados do website discogs (https://www.discogs.com/). Para importação de dados foi usado curl no endpoint deste serviço.

# Salvando os dados em um json


## Membros
Ao longo da carreira os Beatles tiveram 6 membros, apesar dos mais conhecido foram os 4 que ficaram mais tempo e remanesceram até o final da banda. Os membros ativos considerados ativos foram os que fizeram sucesso ao longo da carreira da banda
- John Lennon (Ativo)
- Paul McCartney (Ativo)
- Richard Starkey (Ringo Starr) (Ativo)
- George Harrison (Ativo)
- Pete Best (Inativo)
- Stuart Sutcliffe (Inativo)


In [107]:
!curl https://api.discogs.com/artists/82730 \
| python3 -c "import sys, json; print(json.dumps(json.load(sys.stdin)['members']))" > beatles_members.json


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6546  100  6546    0     0   9311      0 --:--:-- --:--:-- --:--:--  9298


## Releases
São os lançamentos do Beatles entre LP, Singles e etc. A api de dados tem uma limitação de 500 por página, então para este exemplo teremos uma coleção de 500 itens (dentro de muitos que realmente existem)

In [109]:
!curl https://api.discogs.com/artists/82730/releases?per_page=500 \
| python3 -c "import sys, json; print(json.dumps(json.load(sys.stdin)['releases']))" > beatles_releases.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  150k  100  150k    0     0   114k      0  0:00:01  0:00:01 --:--:--  114k


# Importanto membros e releases no banco de dados 'beatles'

In [105]:
!mongoimport --db beatles --collection releases --drop --jsonArray --file $(pwd)/beatles_releases.json

2020-08-15T20:18:11.955+0000	connected to: mongodb://localhost/
2020-08-15T20:18:11.955+0000	dropping: beatles.releases
2020-08-15T20:18:11.988+0000	500 document(s) imported successfully. 0 document(s) failed to import.


In [98]:
!mongoimport --db beatles --collection members --drop --jsonArray --file $(pwd)/beatles_members.json

2020-08-15T20:16:22.003+0000	connected to: mongodb://localhost/
2020-08-15T20:16:22.003+0000	dropping: beatles.members
2020-08-15T20:16:22.044+0000	6 document(s) imported successfully. 0 document(s) failed to import.


# Usando python para consultas

In [81]:
from pymongo import MongoClient
from pprintpp import pprint
import warnings
warnings.filterwarnings('ignore')
mongoclient = MongoClient('localhost', 27017)
db = mongoclient.beatles
print(db)

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'beatles')


# Consultas

## Exemplo de um documento da colection members

In [111]:
db.members.find_one({})

{'_id': ObjectId('5f38429634396553e8ff6d95'),
 'id': 35301,
 'name': 'Paul McCartney',
 'resource_url': 'https://api.discogs.com/artists/35301',
 'active': True}

## Listando todos os membros

In [83]:
result = db.members.find({}, {"_id": 0, "name": 1})
pprint(list(result))

[
    {'name': 'John Lennon'},
    {'name': 'George Harrison'},
    {'name': 'Richard Starkey'},
    {'name': 'Stuart Sutcliffe'},
    {'name': 'Pete Best'},
    {'name': 'Paul McCartney'},
]


## Listando apenas os membros ativos

In [114]:
result = db.members.find({ "active": True }, { "_id": 0 })
pprint( list(result) )

[
    {
        'active': True,
        'id': 35301,
        'name': 'Paul McCartney',
        'resource_url': 'https://api.discogs.com/artists/35301',
    },
    {
        'active': True,
        'id': 46481,
        'name': 'John Lennon',
        'resource_url': 'https://api.discogs.com/artists/46481',
    },
    {
        'active': True,
        'id': 298525,
        'name': 'Richard Starkey',
        'resource_url': 'https://api.discogs.com/artists/298525',
    },
    {
        'active': True,
        'id': 243955,
        'name': 'George Harrison',
        'resource_url': 'https://api.discogs.com/artists/243955',
    },
]


## Exemplo de um documento da collection releases

In [89]:
db.releases.find_one({})

{'_id': ObjectId('5f383a67a7b1af3f7542b46f'),
 'id': 9833515,
 'status': 'Accepted',
 'type': 'release',
 'format': 'Acetate, 10", Mono',
 'label': 'Personal Recording Department',
 'title': 'Hello Little Girl / Till There Was You',
 'resource_url': 'https://api.discogs.com/releases/9833515',
 'role': 'Main',
 'artist': 'The Beatles',
 'year': 1962,
 'thumb': '',
 'stats': {'community': {'in_wantlist': 146, 'in_collection': 8}}}

## Lançamentos do ano de 1963

In [117]:
result = db.releases.find({ "year": 1963 }).distinct("title")
pprint( list(result) )

[
    "All My Loving / Don't Bother Me",
    "All My Loving / It Won't Be Long",
    'All My Loving / Love Me Do',
    'Boys',
    'Ella Te Ama / Quiero Estrechar Tu Mano / La Vi Alla / Seras Mia',
    'From Me To You',
    'From Me To You / I Saw Her Standing There',
    'From Me To You / P. S. I Love You',
    'I Saw Her Standing There',
    'I Saw Her Standing There ',
    'I Saw Her Standing There / Misery',
    'I Want To Hold Your Hand',
    'I Want To Hold Your Hand / Hold Me Tight',
    'I Want To Hold Your Hand / I Saw Her Standing There',
    "It Won't Be Long",
    "Love Me Do / It Won't Be Long",
    'Love Me Do / Please Please Me',
    'Love Me Do / Twist And Shout',
    'Misery',
    'Muchachos = Boys',
    'My Bonnie',
    'My Bonnie / Ya Ya',
    'Not A Second Time / Hold Me Tight',
    'P.S. I Love You / Chains',
    "Please Mr Postman / I'll Get You",
    'Please Please Me',
    'Please Please Me ',
    "Please Please Me / Baby It's You",
    'Please Please Me / From 

## Lançamentos feitos entre 1967 e 1969

In [121]:
result = db.releases.find({ "$and": [ { "year": { "$gte": 1967 } }, { "year": { "$lte": 1969 } } ] }).distinct("title")
pprint( list(result) )


[
    'All You Need Is Love',
    'Bad Boy / Strawberry Fields Forever / Penny Lane / Good Day Sunshine',
    'Christmas Time (Is Here Again)',
    'Hello Goodbye',
    'Hello, Goodbye',
    'Help!',
    'La Banda Del Sargento Pepper',
    'Magical Mystery Tour',
    'Norwegian Wood',
    'Penny Lane',
    "Sgt. Pepper's Lonely Hearts Club Band",
    'Strawberry Fields Forever / Penny Lane',
    'The Early Beatles',
    "This Boy / She's Leaving Home",
    'Yesterday And Today / Beatles VI',
]
